In [70]:
import pandas as pd
import requests
from config import api_key

url="http://api.openweathermap.org/data/2.5/weather"



In [114]:
cities=pd.read_csv("worldcities.csv")

n=3
city=cities["City"]


city=city[::30]

city_list=city.head(500).reset_index(drop=True)

In [115]:
def extract(r):
    return {
        "city": r["City"],
        "lat":
        "lon":
        "humidity":
        "cloudiness":
        "wind_speed(mph)": 
    }

In [117]:
for e in city_list:
    r=requests.get(url=url, params={
        "q": e,
        "appid":api_key
    }).json()

'la laguna del pedernal'